In [ ]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import numpy as np
import json
from torch.utils.data import DataLoader 
from data_manager import load_data, get_data_shapes, get_num_indexes
%load_ext autoreload
%autoreload 2

In [ ]:
X, y, num_games = load_data()

In [ ]:
print("Ilość gier:", num_games)

In [ ]:
get_data_shapes(X,y)

In [ ]:
from dataset import ChessDataset
from model import Model

unique_labels, y_mapped = np.unique(y, return_inverse=True)

int_to_move = {i: int(label) for i, label in enumerate(unique_labels)}
move_to_int = {int(label): i for i, label in enumerate(unique_labels)}

with open("../models/move_mapping.json", "w") as f:
    json.dump(int_to_move, f)


In [ ]:
dataset = ChessDataset(X, y_mapped)

loader = DataLoader(dataset, batch_size=64, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

torch.manual_seed(42)

model = Model(num_of_indexes=len(unique_labels)).to(device)
model

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)  # Raw logits

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(loader):.4f}')

    # Default: => Loss: 0.1763, 6m56s
    # 2x BatchNorm2d(64) and 128 => Loss: 0.1429, 7m23s

In [ ]:
torch.save(model.state_dict(), "../models/TORCH_100EPOCHS.pth")